<center> <h1 style="background-color:DarkSlateBlue; color:white" >Movie Recommendation System</h1> 

![Image](https://www.vshsolutions.com/wp-content/uploads/2020/02/recommender-system-for-movie-recommendation.jpg)

<center>
<br>    
<a id="top"></a>    
<div class="list-group" id="list-tab" role="tablist">
  <h3 class="list-group-item list-group-item-action active" style="background-color:DarkSlateBlue; color:white" data-toggle="list"  role="tab" aria-controls="home">Notebook Content!</h3>  
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#Required libraries" role="tab" aria-controls="profile" style="color:DarkSlateBlue">Required libraries<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">1</span></a>
   <a class="list-group-item list-group-item-action" data-toggle="list" href="#I/O" role="tab" aria-controls="profile" style="color:DarkSlateBlue">I/O<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">2</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#Custom functions" role="tab" aria-controls="profile" style="color:DarkSlateBlue">Custom functions<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">3</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#Data loading" role="tab" aria-controls="profile" style="color:DarkSlateBlue">Data loading<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">4</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#Content based recommender" role="tab" aria-controls="profile" style="color:DarkSlateBlue">Content based recommender<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">5</span></a>

<a id='Required libraries'></a>
<h1 style="color:DarkSlateBlue" >Required libraries</h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In [1]:
#################Libraries##############
#General libraries
import sys

#Data analysis libraries
import pandas as pd
pd.options.display.max_colwidth = 1000
import numpy as np
import operator
import ast
import json
pd.set_option("display.max_columns",100)
pd.set_option("display.max_rows",100)
from tqdm import tqdm_notebook

#Visualization libraries
import plotly
import plotly.express as px
plotly.offline.init_notebook_mode (connected = True)
import ipywidgets as widgets

#Sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity

#Nltk
#nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

#String similarity
from pyjarowinkler import distance as jaro_distance

############################WARNINGS######################
import warnings
warnings.filterwarnings('ignore')

##################################DISPLAY###################################
from IPython.core.display import display, HTML,clear_output, Markdown
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:90% !important; } '
        '.end_space { min-height:0px !important; } '
    '</style>'
))

#from itertools import combinations,product
#!pip install strsimpy
#from strsimpy.normalized_levenshtein import NormalizedLevenshtein

<a id='I/O'></a>
<h1 style="color:DarkSlateBlue" >I/O</h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

#### Input

In [2]:
path_metadata = './dat/movies_metadata.csv'
path_credits = './dat/credits.csv'
path_keywords = './dat/keywords.csv'

#### Output

In [23]:
path_metadata_complete = './dat/movies_metadata_complete.csv'
path_dict_similarity_content_based = './dat/dict_similarity_content_based.json'

<a id='Custom functions'></a>
<h1 style="color:DarkSlateBlue" >Custom functions</h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In [36]:
def get_soup_column(df_metadata,df_credits,df_keywords):
    df_metadata1 = df_metadata.merge(df_credits, on='id',how="inner").drop_duplicates()
    df_metadata_complete = df_metadata1.merge(df_keywords, on='id',how="inner").drop_duplicates()

    #Literal eval
    df_metadata_complete['crew'] = df_metadata_complete['crew'].apply(ast.literal_eval)
    df_metadata_complete['cast'] = df_metadata_complete['cast'].apply(ast.literal_eval)
    df_metadata_complete['keywords'] = df_metadata_complete['keywords'].apply(ast.literal_eval)

    #######################GENRES###########################
    #Let's get the genres of each movie
    df_metadata_complete['genre_formatted'] = df_metadata_complete['genres'].fillna('[]').apply(ast.literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

    ######################DIRECTOR#########################
    #Let's get the directors of each movie
    def get_director(x):
        for i in x:
            if i['job'] == 'Director':
                return i['name']
        return np.nan

    df_metadata_complete['director'] = df_metadata_complete['crew'].apply(get_director) #get diretor from crew column
    df_metadata_complete['director_formatted'] = df_metadata_complete['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", ""))) #lower case and remove white spaces
    df_metadata_complete['director_formatted'] = df_metadata_complete['director_formatted'].apply(lambda x: [x,x]) #mention director twice times to weigh it more

    #####################CAST##############################
    df_metadata_complete['cast_formatted'] = df_metadata_complete['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
    df_metadata_complete['cast_formatted'] = df_metadata_complete['cast_formatted'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
    df_metadata_complete['cast_formatted'] = df_metadata_complete['cast_formatted'].apply(lambda x: x[:3] if len(x) >=3 else x) #keep first 3 actors from the list

    ####################KEYWORDS############################
    #stemmer = SnowballStemmer('english')
    lemmatizer = WordNetLemmatizer()
    df_metadata_complete['keywords_formatted'] = df_metadata_complete['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
    #df_metadata2['keywords_formatted'] = df_metadata2['keywords_formatted'].apply(lambda x: [stemmer.stem(i) for i in x]) #stem of the word
    df_metadata_complete['keywords_formatted'] = df_metadata_complete['keywords_formatted'].apply(lambda x: [lemmatizer.lemmatize(i) for i in x]) #lemma of the word
    df_metadata_complete['keywords_formatted'] = df_metadata_complete['keywords_formatted'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x]) #lowercase and without white spaces

    ##################COMBINE PREVIOUS COLUMNS: GENRE, DIRECTOR, CAST, KEYWORDS#############
    df_metadata_complete['soup'] = df_metadata_complete['genre_formatted'] + df_metadata_complete['director_formatted'] + df_metadata_complete['cast_formatted'] + df_metadata_complete['keywords_formatted']
    df_metadata_complete['soup'] = df_metadata_complete['soup'].apply(lambda x: ' '.join(x))
    return df_metadata_complete

In [4]:
########################STEP 1###############################################
#Tokens
#def processing(df_input,column_identifier,column_similarities):
#    tokens = df_input[[column_identifier, column_similarities]].drop_duplicates(column_similarities).reset_index().drop(columns=['index'])
#    print("Number of different titles:",tokens.shape[0])
#    tokens[column_similarities] = tokens[column_similarities].fillna('NA')
#    indices = pd.Series(tokens.index, index=tokens[column_identifier]).drop_duplicates()
#    return tokens, indices

#########################STEP 2###############################################
#Let's create the vectorizer and the tfidf matrix
def tfidf(df_input, column_similarities, column_identifier):
    vectorizer = TfidfVectorizer(stop_words='english',max_features=None) #It gets the features that will make up the sparse matrix
    tfidf_matrix = vectorizer.fit_transform(df_input[column_similarities])
    #column_names = vectorizer.get_feature_names()
    #df_tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=column_names,index = df_input[column_identifier])
    #n_components = df_tfidf_matrix.shape[1]
    #n_vectors = df_tfidf_matrix.shape[0]
    #print("Tfidf matrix shape:",df_tfidf_matrix.shape)
    #display("Every title is transformed in a vector of {} components, total number of different words. As there are {} distinct titles we have {} distinct vectors".format(n_components,n_vectors,n_vectors),df_tfidf_matrix)
    return tfidf_matrix

########################STEP 3################################################
#Let's create the cosine similarity matrix
def similarities(df_input, tfidf_matrix, column_identifier):
    cosine_sim = cosine_similarity(tfidf_matrix)
    df_cosine_sim = pd.DataFrame(cosine_sim, columns = df_input[column_identifier],index = df_input[column_identifier])
    print("Cosine similarity matrix shape:",df_cosine_sim.shape)
    return df_cosine_sim

In [5]:
def calculate_jaro_distance(*, selected_title, all_possible_titles, num_similarities):
    #Jaro similarity
    similarity_jaro_list = []
    title_names = [title for title in all_possible_titles if str(title)!="nan"]
    for title_name in title_names:
        similarity_jaro = jaro_distance.get_jaro_distance(selected_title.lower(), title_name.lower())
        similarity_jaro_list.append(similarity_jaro)
    titles_dict_jaro = dict(zip(title_names, similarity_jaro_list))
    titles_dict_sorted_jaro = dict(sorted(titles_dict_jaro.items(), key=operator.itemgetter(1),reverse=True)[:num_similarities])
    df_similarities_jaro = pd.DataFrame(list(titles_dict_sorted_jaro.items()),columns = ['title','similarity']) 
    return df_similarities_jaro

In [20]:
def get_most_likely_items_cosine_similarity_dict(*,items,max_number_of_predictions,dict_similarity):
    d = {}
    for movie in items:
        dict_movie = dict_similarity[movie]
        values = list(dict_movie.values())
        columns = list(dict_movie.keys())
        df_movie = pd.DataFrame(data = [values], columns=columns)
        d[movie] = df_movie
    df_similarity_filtered = pd.concat(d.values(), ignore_index=True).fillna(0)
    display(df_similarity_filtered)
    df_most_similar_items = df_similarity_filtered.sum(axis = 0).reset_index().rename(columns={'index':'title', 0:'similarity'}).sort_values(by="similarity",ascending=False)
    fig = px.bar(df_most_similar_items.head(max_number_of_predictions), x="title",y="similarity",title="Recommended movies", 
           labels={'similarity': "Similarity"}, height=500)
    fig.show()

In [15]:
def get_most_likely_items_cosine_similarity_matrix(*,items,max_number_of_predictions,df_similarity):
    df_similarity_filtered = df_similarity[df_similarity['title'].isin(items)].drop(columns=items)
    display(df_similarity_filtered)
    df_most_similar_items = df_similarity_filtered.drop(columns=['title']).sum(axis = 0).reset_index().rename(columns={0:'similarity'}).sort_values(by="similarity",ascending=False)
    fig = px.bar(df_most_similar_items.head(max_number_of_predictions), x="title",y="similarity",title="Recommended movies", 
           labels={'similarity': "Similarity"}, height=500)
    fig.show()

<a id='Data loading'></a>
<h1 style="color:DarkSlateBlue" >Data loading</h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In [8]:
df_metadata = pd.read_csv(path_metadata,low_memory=False)
df_metadata['id'] = df_metadata['id'].astype('str')
df_metadata['revenue'] = df_metadata['revenue'].astype('float')
df_metadata['imdb_id'] = df_metadata['imdb_id'].str.replace('tt','')
df_metadata = df_metadata[df_metadata['revenue']>10_000_000].drop_duplicates('title')
display(df_metadata.head(1))

#Credits dataset
df_credits = pd.read_csv(path_credits)
df_credits['id'] = df_credits['id'].astype('str')

#Keywords dataset
df_keywords = pd.read_csv(path_keywords)
df_keywords['id'] = df_keywords['id'].astype('str')

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",http://toystory.disney.com/toy-story,862,0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [9]:
#Image that we will use in the notebook
#response = requests.get("https://wpamelia.com/wp-content/uploads/2019/06/loading1.jpg")
#image = Image.open(BytesIO(response.content)) 

<a id='Content based recommender'></a>
<h1 style="color:DarkSlateBlue"> Content based recommender </h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

Previously, we built a recommender that only took the title into consideration. Now we are gonna build a content based recommender that takes **genre**, **director**, **cast** and **keywords** into consideration.

From now on, in order to reduce the computation cost, we will be analysing just the movies that had **at least 1 million dollars in revenues**.

In [43]:
df_metadata_complete = get_soup_column(df_metadata, df_credits, df_keywords)
columns_selected = ['title','release_date','director','budget','revenue','production_companies','production_countries',
                   'genres','runtime','spoken_languages','popularity','vote_average','vote_count','soup']
df_metadata_complete[columns_selected].sort_values(by="revenue",ascending=False).to_csv(path_metadata_complete, sep=",",index=False)

In [44]:
df_metadata_complete[columns_selected].sort_values(by="revenue",ascending=False)

,title,release_date,director,budget,revenue,production_companies,production_countries,genres,runtime,spoken_languages,popularity,vote_average,vote_count,soup
2988,Avatar,2009-12-10,James Cameron,237000000,2.787965e+09,"[{'name': 'Ingenious Film Partners', 'id': 289}, {'name': 'Twentieth Century Fox Film Corporation', 'id': 306}, {'name': 'Dune Entertainment', 'id': 444}, {'name': 'Lightstorm Entertainment', 'id': 574}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}, {'iso_3166_1': 'GB', 'name': 'United Kingdom'}]","[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 878, 'name': 'Science Fiction'}]",162.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'es', 'name': 'Español'}]",185.070892,7.2,12114.0,Action Adventure Fantasy Science Fiction jamescameron jamescameron samworthington zoesaldana sigourneyweaver cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d
3808,Star Wars: The Force Awakens,2015-12-15,J.J. Abrams,245000000,2.068224e+09,"[{'name': 'Lucasfilm', 'id': 1}, {'name': 'Truenorth Productions', 'id': 1634}, {'name': 'Bad Robot', 'id': 11461}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]","[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 878, 'name': 'Science Fiction'}, {'id': 14, 'name': 'Fantasy'}]",136.0,"[{'iso_639_1': 'en', 'name': 'English'}]",31.626013,7.5,7993.0,Action Adventure Science Fiction Fantasy j.j.abrams j.j.abrams daisyridley johnboyega adamdriver android spaceship jedi imax spaceopera 3d
526,Titanic,1997-11-18,James Cameron,200000000,1.845034e+09,"[{'name': 'Paramount Pictures', 'id': 4}, {'name': 'Twentieth Century Fox Film Corporation', 'id': 306}, {'name': 'Lightstorm Entertainment', 'id': 574}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]","[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}, {'id': 53, 'name': 'Thriller'}]",194.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}, {'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso_639_1': 'sv', 'name': 'svenska'}, {'iso_639_1': 'it', 'name': 'Italiano'}, {'iso_639_1': 'ru', 'name': 'Pусский'}]",26.88907,7.5,7770.0,Drama Romance Thriller jamescameron jamescameron katewinslet leonardodicaprio francesfisher shipwreck iceberg ship panic titanic oceanliner epic richwoman-poorman love disaster tragiclove classdifferences imax starcrossedlovers steerage salvage richsnob 3d 1910s
3766,Jurassic World,2015-06-09,Colin Trevorrow,150000000,1.513529e+09,"[{'name': 'Universal Studios', 'id': 13}, {'name': 'Amblin Entertainment', 'id': 56}, {'name': 'Legendary Pictures', 'id': 923}, {'name': 'Fuji Television Network', 'id': 3341}, {'name': 'Dentsu', 'id': 6452}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]","[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 878, 'name': 'Science Fiction'}, {'id': 53, 'name': 'Thriller'}]",124.0,"[{'iso_639_1': 'en', 'name': 'English'}]",32.790475,6.5,8842.0,Action Adventure Science Fiction Thriller colintrevorrow colintrevorrow chrispratt brycedallashoward irrfankhan monster dna tyrannosaurusrex velociraptor island sequel suspense disaster escape dinosaur amusementpark animalattack themepark jurassicpark 3d animalhorror
3853,Furious 7,2015-04-01,James Wan,190000000,1.506249e+09,"[{'name': 'Universal Pictures', 'id': 33}, {'name': 'Original Film', 'id': 333}, {'name': 'Fuji Television Network', 'id': 3341}, {'name': 'Dentsu', 'id': 6452}, {'name': 'One Race Films', 'id': 7154}, {'name': 'China Film Co.', 'id': 40890}, {'name': 'Québec Production Services Tax Credit', 'id': 86352}, {'name': 'Media Rights Capital (MRC)', 'id': 86655}, {'name': 'Abu Dhabi Film Commission', 'id': 87857}, {'name': 'Colorado Office of Film, Television & Media', 'id': 87858}]","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_3

#Filtering keywords that only appear once
s = df_metadata2.apply(lambda x: pd.Series(x['keywords_formatted']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword_formatted'
s = s.value_counts()

s = s[s > 1]

def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

df_metadata2['keywords_formatted'] = df_metadata2['keywords_formatted'].apply(filter_keywords)

#from nltk.corpus import stopwords
#stop = stopwords.words('english')
#tokens['title_clean'] = tokens['title'].apply(lambda x: x.lower()).apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))

In [25]:
###########Let's call the functions
#tokens_content_based, indices_content_based = processing(df_input = df_metadata_complete, column_identifier = 'title', column_similarities = 'soup')
tfidf_matrix_content_based= tfidf(df_input = df_metadata_complete, column_similarities = 'soup', column_identifier = 'title')
cosine_sim_content_based = similarities(df_input = df_metadata_complete, tfidf_matrix = tfidf_matrix_content_based, column_identifier = 'title').reset_index()
print("Size of matrix:",sys.getsizeof(cosine_sim_content_based)/10**6 ,"MB")
display(cosine_sim_content_based)

Cosine similarity matrix shape: (4271, 4271)
Size of matrix: 146.243233 MB


title,title,Toy Story,Jumanji,Waiting to Exhale,Father of the Bride Part II,Heat,Sudden Death,GoldenEye,The American President,Balto,Nixon,Cutthroat Island,Casino,Sense and Sensibility,Ace Ventura: When Nature Calls,Money Train,Get Shorty,Assassins,Leaving Las Vegas,Now and Then,Dangerous Minds,Twelve Monkeys,Babe,Dead Man Walking,Mortal Kombat,To Die For,How To Make An American Quilt,Se7en,Pocahontas,The Usual Suspects,Home for the Holidays,Mr. Holland's Opus,Friday,From Dusk Till Dawn,Fair Game,Bed of Roses,White Squall,Mary Reilly,Vampire in Brooklyn,Broken Arrow,Happy Gilmore,The Bridges of Madison County,Muppet Treasure Island,Braveheart,Taxi Driver,Rumble in the Bronx,Boomerang,Flirting with Disaster,The Birdcage,Bad Boys,...,The Shack,Power Rangers,Alien: Covenant,CHiPS,Going in Style,RRRrrrr!!!,Smurfs: The Lost Village,The Zookeeper's Wife,Gifted,24,Diary of a Wimpy Kid: The Long Haul,Baywatch,Unforgettable,Snatched,The Fate of the Furious,Captain Underpants: The First Epic Movie,Cars 3,In This Corner of the World,How to Be a Latin Lover,Nasha Russia: Yaytsa sudby,Baahubali 2: The Conclusion,The Irony of Fate. The Sequel,Kabhi Alvida Naa Kehna,The Beguiled,Baby Driver,Rough Night,Weekend Pass,Boj S Tenyu 2: Revansh,Lovey-Dovey 2,Love and the City 2,High Security Vacation,Despicable Me 3,War for the Planet of the Apes,Kidnap,Valerian and the City of a Thousand Planets,Mudhalvan,Fanaa,Atomic Blonde,Dunkirk,Bairavaa,Confidential Assignment,Transformers: The Last Knight,"Mommies, Happy New Year!",Good Time,The Dark Tower,My Old Classmate,The Emoji Movie,Wind River,Baasha,Sivaji: The Boss
0,Toy Story,1.000000,0.010876,0.005905,0.004970,0.000000,0.000000,0.000000,0.005687,0.024768,0.000000,0.000000,0.000000,0.000000,0.005279,0.005685,0.004460,0.035711,0.000000,0.015012,0.000000,0.000000,0.013369,0.000000,0.000000,0.005326,0.000000,0.000000,0.027189,0.000000,0.005435,0.010276,0.006009,0.000000,0.000000,0.000000,0.000000,0.000000,0.006600,0.000000,0.005958,0.000000,0.017064,0.000000,0.000000,0.004718,0.004818,0.004765,0.004824,0.003568,...,0.000000,0.000000,0.000000,0.004860,0.005091,0.006401,0.040055,0.000000,0.000000,0.000000,0.020738,0.005246,0.000000,0.005140,0.000000,0.034818,0.038148,0.019414,0.005283,0.0,0.000000,0.006808,0.000000,0.000000,0.000000,0.005215,0.004954,0.000000,0.006015,0.006032,0.006110,0.041439,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004476,0.000000,0.006202,0.000000,0.000000,0.000000,0.035808,0.000000,0.000000,0.005326
1,Jumanji,0.010876,1.000000,0.000000,0.000000,0.000000,0.009607,0.006699,0.000000,0.017143,0.000000,0.009397,0.000000,0.000000,0.009007,0.000000,0.000000,0.007033,0.000000,0.010343,0.000000,0.000000,0.019929,0.000000,0.011827,0.013693,0.000000,0.000000,0.011690,0.000000,0.000000,0.010288,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006118,0.000000,0.000000,0.020834,0.000000,0.000000,0.000000,0.000000,0.000000,0.043209,0.000000,...,0.012253,0.008162,0.000000,0.000000,0.000000,0.000000,0.023560,0.000000,0.000000,0.000000,0.014289,0.000000,0.000000,0.000000,0.000000,0.011557,0.022439,0.000000,0.000000,0.0,0.009582,0.000000,0.000000,0.052612,0.000000,0.000000,0.000000,0.000000,0.015464,0.000000,0.000000,0.024374,0.000000,0.000000,0.005773,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010851,0.000000,0.000000,0.014630,0.000000,0.011885,0.000000,0.000000,0.000000
2,Waiting to Exhale,0.005905,0.000000,1.000000,0.005962,0.003650,0.000000,0.000000,0.024772,0.000000,0.005666,0.000000,0.005898,0.034729,0.006334,0.006820,0.026366,0.000000,0.013049,0.010290,0.004239,0.000000,0.009164,0.003370,0.000000,0.011709,0.013846,0.000000,0.009422,0.004935,0.023674,0.004650,0.007209,0.000000,0.019541,0.018255,0.004019,0.016387,0.118863,0.003581,0.007148,0.012503,0.006383,0.004528,0.004671,0.005660,0.016176,0.015998,0.005787,0.004281,...,0.027218,0.000000,0.000000,0.005831,0.011192,0.007679,0.007218,0.005033,0.004349,0.000000,0.007758,0.006294,0.0058

In [12]:
%%time
dict_similarity = {}
number_of_similarities_per_title = 50
for title in tqdm_notebook(cosine_sim_content_based['title'].unique()):
    dict_similarity[title] = dict(cosine_sim_content_based[cosine_sim_content_based['title']==title].iloc[0].iloc[1:])
    dict_similarity[title] = dict(sorted(dict_similarity[title].items(), key=operator.itemgetter(1),reverse=True)[1:number_of_similarities_per_title])
print("Size of matrix:",sys.getsizeof(dict_similarity)/10**6 ,"MB")

#Saving the dictionary as a json file
with open(path_dict_similarity_content_based, 'w') as fp:
    json.dump(dict_similarity, fp)

  0%|          | 0/4270 [00:00<?, ?it/s]

Size of matrix: 0.14756 MB
Wall time: 1min 33s


In [13]:
#with open(path_dict_similarity_content_based, 'r') as fp:
#    dict_similarity = json.load(fp)

In [32]:
#Let's plot the results
# Button, text box, output
layout = widgets.Layout(width='400px', height='25px') #set width and height
butt_cosine_content = widgets.Button(description='Display similar titles based on cosine similarity',layout = layout,button_style='success')
items_cosine_content = widgets.Text(value='Toy Story',description='Title',layout=layout)
num_similar_movies_cosine_content = widgets.Dropdown(options=list(range(1,21)),value=10,description='Num similar movies',disabled=False,layout=layout)
output_cosine_content = widgets.Output()

def on_butt_clicked(b):
    with output_cosine_content:
        clear_output()
        itemset = list(items_cosine_content.value.split(","))
        itemset = [item.strip().title() for item in itemset]
        print(itemset)
        items_no_reconocidos = [item for item in itemset if item not in list(dict_similarity.keys())]
        if len(items_no_reconocidos)==0:
            #################################ITEM-TO-ITEM COLLABORATIVE FILTERING USING COSINE SIMILARITIES##########################
            #get_most_likely_items_cosine_similarity_matrix(items = itemset, max_number_of_predictions = num_similar_movies_cosine_content.value, df_similarity = cosine_sim_content_based)
            get_most_likely_items_cosine_similarity_dict(items = itemset, max_number_of_predictions = num_similar_movies_cosine_content.value, dict_similarity = dict_similarity)
        else:
            if len(items_no_reconocidos)==1:
                print(f"The title {items_no_reconocidos} is not recognised", "\n")
            if len(items_no_reconocidos)>1:
                print(f"The titles {items_no_reconocidos} are not recognised", "\n")
            for item in items_no_reconocidos:
                df_similarities_jaro = calculate_jaro_distance(selected_title = item, all_possible_titles = df_metadata_complete['title'].unique(), num_similarities = 1)
                print("Maybe you meant:",df_similarities_jaro['title'].values[0])
                
butt_cosine_content.on_click(on_butt_clicked)
widgets.VBox([butt_cosine_content,
              items_cosine_content,
              num_similar_movies_cosine_content,
              output_cosine_content])  